In [503]:
import pandas as pd
import numpy as np 
import imageio as iio
from PIL import  Image
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential 
from keras.optimizers import Adam
import keras
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Input, Dropout
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split


In [504]:
# Define the amount of pictures you want to load in
MAX_PICTURES = 10015

In [505]:
df = pd.read_csv('merged.csv', index_col=0)

In [506]:
df = df.sample(frac=1).reset_index(drop=True)

In [507]:
# read first 100 images from dataframe 
image_list = []
for image in df['image_path'][0:MAX_PICTURES]:
    im = Image.open(image)
    im = im.resize((100,75))
    a = np.asarray(im)
    image_list.append(a)


In [508]:
df

,lesion_id,image_id,dx,dx_type,age,sex,localization,image_path
0,HAM_0005689,ISIC_0034229,nv,consensus,20.0,male,back,data/ham10000_images_part_2\ISIC_0034229.jpg
1,HAM_0003825,ISIC_0030656,nv,follow_up,30.0,female,abdomen,data/ham10000_images_part_2\ISIC_0030656.jpg
2,HAM_0000113,ISIC_0027989,bkl,histo,75.0,female,hand,data/ham10000_images_part_1\ISIC_0027989.jpg
3,HAM_0003180,ISIC_0031058,nv,follow_up,40.0,male,upper extremity,data/ham10000_images_part_2\ISIC_0031058.jpg
4,HAM_0000798,ISIC_0031064,nv,histo,65.0,male,foot,data/ham10000_images_part_2\ISIC_0031064.jpg
...,...,...,...,...,...,...,...,...
10010,HAM_0000893,ISIC_0032303,bkl,histo,70.0,male,lower extremity,data/ham10000_images_part_2\ISIC_0032303.jpg
10011,HAM_0001684,ISIC_0028576,nv,histo,30.0,female,trunk,data/ham10000_images_part_1\ISIC_0028576.jpg
10012,HAM_0006105,ISIC_0032236,nv,histo,40.0,female,foot,data/ham10000_images_part_2\ISIC_0032236.jpg
10013,HAM_0002003,ISIC_0031934,nv,histo,20.0,male,back,data/ham10000_images_part_2\ISIC_0031934.jpg


In [509]:
mel_dict = {
    'akiec': 'Actinic keratosis-like lesions',
    'bcc': 'basal cell carcinoma',
    'bkl': 'benign keratosis',
    'df': 'dermatofibroma',
    'mel': 'melanoma',
    'nv': 'melanocytic nevi',
    'vasc': 'vascular lesions / hemorrhage'
}

In [510]:
y = df['dx'][0:MAX_PICTURES]
y

0         nv
1         nv
2        bkl
3         nv
4         nv
        ... 
10010    bkl
10011     nv
10012     nv
10013     nv
10014     nv
Name: dx, Length: 10015, dtype: object

In [511]:
# map diagnoses to dictionary
mel_dict = {
    'akiec': 0,
    'bcc': 1,
    'bkl': 2,
    'df': 3,
    'mel': 4,
    'nv': 5,
    'vasc': 6
}

In [512]:
image_list = np.asarray(image_list)
y = y.map(mel_dict)
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(image_list, y, test_size=0.20, random_state=42)


In [513]:
# ALWAYS CONVERT AN ARRAY TO AN NUMPY ARRAY
y = np.asarray(y) 

In [514]:
y.shape

(10015, 7)

In [515]:
input_shape = (75, 100, 3)
num_classes = 7

model = Sequential()
model.add(Conv2D(75, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
model.add(Conv2D(75, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [516]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [517]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_64 (Conv2D)          (None, 75, 100, 75)       2100      
                                                                 
 conv2d_65 (Conv2D)          (None, 73, 98, 75)        50700     
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 36, 49, 75)       0         
 g2D)                                                            
                                                                 
 dropout_48 (Dropout)        (None, 36, 49, 75)        0         
                                                                 
 conv2d_66 (Conv2D)          (None, 34, 47, 64)        43264     
                                                                 
 conv2d_67 (Conv2D)          (None, 32, 45, 64)        36928     
                                                     

In [518]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

c:\Users\ozcan\.pyenv\pyenv-win\versions\3.10.7\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [519]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [520]:
epochs = 20
batch_size = 10


model.fit(x=X_train, y=y_train, epochs=epochs, batch_size=batch_size, callbacks=[learning_rate_reduction], validation_data=(X_test, y_test))

Epoch 1/10
 924/1603 [================>.............] - ETA: 1:39 - loss: 1.8431 - accuracy: 0.6539

KeyboardInterrupt: 

In [ ]:
model.evaluate(x=X_test, y=y_test)

7/7 [==============================] - 1s 151ms/step - loss: 1.0310 - accuracy: 0.7000


[1.0310076475143433, 0.699999988079071]